In [1]:
print("123")

123


In [ ]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

headers = {
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "referer": "https://www.card-gorilla.com/"
}

def get_super_detailed_info(card_idx):
    """HTML 태그가 섞인 초정밀 혜택(실적별 한도 등)을 텍스트로 추출"""
    url = f"https://api.card-gorilla.com:8080/v1/cards/{card_idx}"
    try:
        res = requests.get(url, headers=headers)
        card_data = res.json()
        
        # 'key_benefit' 섹션 접근
        key_benefits = card_data.get('key_benefit', [])
        all_details = []
        
        for kb in key_benefits:
            # 실적별 한도와 유의사항이 포함된 'info' 필드 추출
            html_content = kb.get('info', '')
            if html_content:
                # BeautifulSoup으로 깨끗한 텍스트로 변환
                soup = BeautifulSoup(html_content, 'html.parser')
                clean_text = soup.get_text(separator=" ", strip=True)
                all_details.append(f"[{kb.get('title')}]: {clean_text}")
        
        return " \n ".join(all_details)
    except:
        return "정보 수집 실패"

# --- 데이터 수집 루프 시작 ---
card_info_list = []
page = 1

print("🚀 전체 카드 데이터 수집을 시작합니다. (상세 내용 포함)")

while True:
    # 1. 현재 페이지의 카드 리스트 가져오기
    list_url = f"https://api.card-gorilla.com:8080/v1/cards?p={page}&perPage=20&cate=CRD"
    response = requests.get(list_url, headers=headers)
    data = response.json()
    goods_list = data.get("data", [])

    # 2. 만약 더 이상 수집할 데이터가 없으면 루프 종료
    if not goods_list:
        print(f"\n✅ 수집 완료! 총 {len(card_info_list)}개의 카드를 수집했습니다.")
        break

    print(f"\n📦 현재 {page}페이지 수집 중... (누적: {len(card_info_list)}건)")

    # 3. 리스트에 있는 개별 카드의 상세 정보 수집
    for card in goods_list:
        card_idx = card.get("idx")
        card_name = card.get("name")
        
        # 초정밀 정보 가져오기 루틴
        super_detail = get_super_detailed_info(card_idx)
        
        row = {
            "카드ID": card_idx,
            "카드명": card_name,
            "카드사": card.get("corp_txt"),
            "전월실적기준": card.get("pre_month_money"),
            "연회비": card.get("annual_fee_basic"),
            "초정밀혜택": super_detail
        }
        card_info_list.append(row)
        
        # 서버 매너를 위해 0.5초 대기 (차단 방지)
        time.sleep(0.5)

    page += 1 # 다음 페이지로 이동

# --- 결과 저장 ---
df_final = pd.DataFrame(card_info_list)
# 엑셀 깨짐 방지를 위해 utf-8-sig로 저장
df_final.to_csv("card_ultra_details_all.csv", index=False, encoding="utf-8-sig")

print("\n✨ 모든 데이터가 'card_ultra_details_all.csv'로 저장되었습니다.")